In [38]:
import csv
import pandas as pd
    #Read the lvm file
with open('13001.lvm', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    raw_data = list(csv_reader)
    # -------------------------------
    # Step 1: Flatten into a list of strings
    # -------------------------------
lines = []
for item in raw_data:
    if item:  # make sure it's not empty
        lines.append(item[0])  # take the string inside
    # -------------------------------
    # Step 2: Find the line with the last End_of_Header
    # -------------------------------
end_header_indices = []
for i, line in enumerate(lines):
    if "***End_of_Header***" in line:
        end_header_indices.append(i)
start_idx = end_header_indices[-1] + 1  # the line after header

    # -------------------------------
    # Step 3: Extract header and data
    # -------------------------------
header_line = lines[start_idx]
header = header_line.split("\t")
    
data_lines = []
for line in lines[start_idx + 1:]:
    if line.strip():  # skip empty lines
        values = line.split("\t")
            # Pad with empty values if too short
        while len(values) < len(header):
            values.append("")
        data_lines.append(values)
    
    # -------------------------------
    # Step 4: Build DataFrame
    # -------------------------------
df = pd.DataFrame(data_lines, columns=header)
    
    # Convert to numeric where possible
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except:
        pass
            
df = df.set_index("X_Value").iloc[:, :-1]      

In [1]:
import pandas as pd

In [11]:
# 先找最后一个 End_of_Header 的行号
with open('13001.lvm', 'r') as f:
    lines = f.readlines()
start_idx = max(i for i, line in enumerate(lines) if "***End_of_Header***" in line) + 1

# 用 pandas 读数据（跳过 header 前的行）
df = pd.read_csv('13001.lvm', sep="\t", skiprows=start_idx)

# 转换为数值（非数值自动忽略）
df = df.apply(pd.to_numeric)

# 设置 index，并丢掉最后一列（通常是空列）
df = df.drop(df.columns[-1], axis=1)

In [12]:
df

,X_Value,Moog Foce_kN,Moog Stroke_mm,LVDT 1_mm,LVDT 2_mm,LVDT 3_mm,LVDT 4_mm,LVDT 5_mm,LVDT 6_mm
0,0.0,0.101575,-237.612941,-0.002635,-0.000358,-0.018381,-0.002301,-0.001445,-0.004610
1,0.5,0.102648,-237.612083,0.003265,0.006615,-0.011407,-0.000155,0.001237,-0.004789
2,1.0,0.102380,-237.615516,0.008093,0.005274,-0.001752,-0.006324,-0.004127,-0.007650
3,1.5,0.103989,-237.612727,0.008093,0.012784,-0.012212,-0.002837,-0.005200,-0.004253
4,2.0,0.101665,-237.615838,0.003265,0.007420,-0.014626,-0.007129,-0.002249,-0.004610
...,...,...,...,...,...,...,...,...,...
506,253.0,6.641427,-166.121552,-59.983270,-60.328875,-83.967505,-84.973078,-88.892539,-91.772821
507,253.5,6.648401,-166.119728,-59.977101,-60.328875,-83.978234,-84.975760,-88.889320,-91.774073
508,254.0,6.656895,-166.122089,-59.982197,-60.327802,-83.971528,-84.976297,-88.889857,-91.771748
509,254.5,6.656716,-166.122625,-59.986489,-60.332899,-83.971797,-84.981929,-88.890930,-91.773536


In [49]:
df['Moog Foce_kN'][1]

np.float64(0.10238)

In [50]:
type(df['Moog Foce_kN'][1])

numpy.float64

In [3]:
# 先找最后一个 End_of_Header 的行号
with open('13001.lvm', 'r') as f:
    lines = f.readlines()
start_idx = max(i for i, line in enumerate(lines) if "***End_of_Header***" in line) + 1

# 用 pandas 读数据（跳过 header 前的行）
df = pd.read_csv('13001.lvm', sep="\t", skiprows=start_idx)

# 转换为数值（非数值自动忽略）
df = df.apply(pd.to_numeric)

# 设置 index，并丢掉最后一列（通常是空列）
df = df.drop(df.columns[-1], axis=1)
# set index for data
force_index = 2
displ_index = 3
lvdt_1_index = 4
lvdt_2_index = 5
lvdt_3_index = 6
lvdt_4_index = 7
lvdt_5_index = 8
lvdt_6_index = 9

force = df[df.columns[force_index-1]]
displ = df[df.columns[displ_index-1]]
lvdt_1 = df[df.columns[lvdt_1_index-1]]
lvdt_2 = df[df.columns[lvdt_2_index-1]]
lvdt_3 = df[df.columns[lvdt_3_index-1]]
lvdt_4 = df[df.columns[lvdt_4_index-1]]
lvdt_5 = df[df.columns[lvdt_5_index-1]]
lvdt_6 = df[df.columns[lvdt_6_index-1]]

df_new = pd.concat([force, displ, lvdt_1, lvdt_2, lvdt_3, lvdt_4, lvdt_5, lvdt_6],axis = 1)
df_new.columns = ['F','D','D1','D2','D3','D4','D5','D6']

In [39]:
import pandas as pd

def beam_read_lvm_file(file_name, force_index=2, displ_index=3,
             mid_lvdt_ind=None, left_lvdt_ind=None, right_lvdt_ind=None):
    """
    Read a .lvm file and extract force, displacement, and LVDT data in groups.

    Parameters
    ----------
    file_name : str
        Path to the .lvm file to read.
    force_index : int, default=2
        1-based column index for the force data.
    displ_index : int, default=3
        1-based column index for the displacement (stroke) data.
    mid_lvdt_ind : int or list of int, optional
        Column index/indices for the middle LVDTs.
    left_lvdt_ind : int or list of int, optional
        Column index/indices for the left LVDTs.
    right_lvdt_ind : int or list of int, optional
        Column index/indices for the right LVDTs.

    Returns
    -------
    df_new : pandas.DataFrame
        Extracted data as a DataFrame with columns renamed:
        ['Force (kN)', 'Stroke (mm)', 'Mid_1 (mm)', 'Mid_2 (mm)',
         'Left_1 (mm)', 'Left_2 (mm)', 'Right_1 (mm)', 'Right_2 (mm)'].
        Only existing columns are included; missing columns are ignored.
    file_base_name : str
        Base name of the file without the '.lvm' extension.

    Notes
    -----
    - Automatically detects the start of the data by locating the last
      occurrence of '***End_of_Header***' in the file.
    - Converts all columns to numeric values; non-numeric entries
      are coerced to NaN.
    - The last column is dropped assuming it is empty.
    - Single integer indices are automatically converted to lists.
    - LVDT columns are combined in the order: mid -> left -> right.
    """
    
    mid_lvdt_ind = mid_lvdt_ind or []
    left_lvdt_ind = left_lvdt_ind or []
    right_lvdt_ind = right_lvdt_ind or []
    
    if isinstance(mid_lvdt_ind, int):
        mid_lvdt_ind = [mid_lvdt_ind]
    if isinstance(left_lvdt_ind, int):
        left_lvdt_ind = [left_lvdt_ind]
    if isinstance(right_lvdt_ind, int):
        right_lvdt_ind = [right_lvdt_ind]
    
    lvdt_indices = mid_lvdt_ind + left_lvdt_ind + right_lvdt_ind
    
    with open(file_name, 'r') as f:
        lines = f.readlines()
    start_idx = max(i for i, line in enumerate(lines) if "***End_of_Header***" in line) + 1
    df = pd.read_csv(file_name, sep="\t", skiprows=start_idx)
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.drop(df.columns[-1], axis=1)
    
    user_indices = [force_index, displ_index] + lvdt_indices
    existing_cols = [df.columns[i-1] for i in user_indices if i-1 < len(df.columns)]
    
    df_new = df[existing_cols].copy()
    
    col_names = ['Force (kN)','Stroke (mm)','Mid_1 (mm)','Mid_2 (mm)','Left_1 (mm)','Left_2 (mm)','Right_1 (mm)','Right_2 (mm)']
    df_new.columns = col_names[:len(existing_cols)]
    
    return df_new, file_name[:-4]

In [40]:
final  = beam_read_lvm_file('13001.lvm',2,3,[4,5],[7,8],[6,9])

In [41]:
final[0]['Force (kN)']

0      0.101575
1      0.102648
2      0.102380
3      0.103989
4      0.101665
         ...   
506    6.641427
507    6.648401
508    6.656895
509    6.656716
510    6.659487
Name: Force (kN), Length: 511, dtype: float64

In [42]:
import struct_post as sp

In [43]:
four_point_bending(final, 63,150,1300)

Sample Name: 13001
----------------------------------------


('13001',
 {'E_app': np.float64(4220.56764516983),
  'E_true': np.float64(-7364.803855754146)})

In [19]:
from pandas import DataFrame
def four_point_bending (data: DataFrame, 
                        width: float, 
                        depth: float, 
                        beam_span: float):
    """
    Perform four-point bending analysis to calculate apparent and true modulus of elasticity.

    Parameters
    ----------
    data : tuple
        A tuple containing:
        - data[0] : pandas.DataFrame
            Experimental dataset with columns:
            - 'Moog Force_kN' : Applied load (kN).
            - 'LVDT 1_mm' ... 'LVDT 6_mm' : Deflections from six LVDTs (mm).
        - data[1] : str
            Sample name identifier.
    width : float
        Beam specimen width (mm).
    depth : float
        Beam specimen depth (mm).
    beam_span : float
        Beam span length (mm).

    Returns
    -------
    tuple
        (sample_name, results) where:
        - sample_name : str
            Name of the processed sample.
        - results : dict
            Dictionary containing:
            - "E_app" : float
                Apparent modulus of elasticity (MPa).
            - "E_true" : float
                True modulus of elasticity (MPa).

    Notes
    -----
    - Apparent modulus (E_app) is calculated from mid-span deflection (LVDT 3 & 4).
    - True modulus (E_true) is calculated from relative deflection (mid-span vs. supports).
    - Load range for regression is limited to 10–40% of ultimate load.
    """
    
    import pandas as pd
    import numpy as np

    #Experimental test data post-process
    sample_name = data[1]
    force = data[0]['Force (kN)'] * 1000
    delta_1 = abs(data[0]['Left_1 (mm)'])
    delta_2 = abs(data[0]['Left_2 (mm)'])
    delta_3 = abs(data[0]['Mid_1 (mm)'])
    delta_4 = abs(data[0]['Mid_2 (mm)'])
    delta_5 = abs(data[0]['Right_1 (mm)'])
    delta_6 = abs(data[0]['Right_2 (mm)'])

    F_ult = force.max()
    f_b = (F_ult * beam_span) / (width * depth **2) #MPa

    
    delta_ms = (delta_3 + delta_4)/2
    delta_rel = delta_ms - (delta_1 + delta_2 + delta_5 + delta_6) / 4
    
    
    lower_bound = 0.1 * F_ult
    upper_bound = 0.4 * F_ult

    calcs_reg = (lower_bound <= force) & (force <= upper_bound)
    
    F_ms = force[calcs_reg]
    delta_ms_calcs = delta_ms[calcs_reg]
    delta_rel_calcs = delta_rel[calcs_reg]
    
    Delta_ms, intercept_ms = np.polyfit(delta_ms_calcs,F_ms,1)
    Delta_rel, intercept_rel = np.polyfit(delta_rel_calcs,F_ms,1)
    
    E_app = (23/108) * (beam_span/depth)**3 * Delta_ms * (1/width)
    E_true = (1/36) *  (beam_span/depth)**3 * Delta_rel * (1/width)
    
    results = {
        "E_app": E_app,
        "E_true": E_true,
    }

    print(f"Sample Name: {sample_name}")
    print('-' * 40)
    return sample_name, results
